# Leaf Classification CNN

## Libraries

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#%tensorflow_version 2.x

import tensorflow as tf
import numpy as np
import cv2
import sys, os, shutil, glob, random, csv, time
import matplotlib.pyplot as plt
from IPython.display import clear_output
%matplotlib inline
from sklearn.model_selection import train_test_split

## Data IO / Image Preprocessing

In [ ]:
#IO functions

def load_image(file_path):
    return cv2.imread(file_path)

def create_directory(dirname):
    try:
        os.mkdir(dirname)
    except FileExistsError:
        pass

def extract_names(label_file):
    names = list()
    with open('label_legend.csv', newline='') as f:
        reader = csv.reader(f)
        legend = dict(reader)
    for img in label_file:
        names.append(legend[str(img)])
    return names

def process_data(source_path, dest_path, desired_size, transform=False, edges=False):
    create_directory(dest_path)
    names = []
    legend = dict()
    j=0
    species_folders = os.listdir(source_path)
    for i, folder in enumerate(species_folders):
        if folder[0] == '.':
            continue
        id = str(i).rjust(2, '0')
        legend[folder] = id
        folder_path = os.listdir(os.path.join(source_path, folder))
        num_folders = len(folder_path)
        clear_output(wait=True)
        print("Folder:", i, "/100", flush=True)
        for jpgfile in folder_path:
            #resize
            img = load_image(os.path.join(source_path, folder, jpgfile))
            old_size = img.shape[:2]
            ratio = float(desired_size)/max(old_size)
            new_size = tuple([int(x*ratio) for x in old_size])
            img = cv2.resize(img, (new_size[1], new_size[0]))
            delta_w = desired_size - new_size[1]
            delta_h = desired_size - new_size[0]
            top, bottom = delta_h//2, delta_h-(delta_h//2)
            left, right = delta_w//2, delta_w-(delta_w//2)

            color = [0, 0, 0]
            new_im = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT,
                value=color)
    
            #rename
            new_name = id + str(j)
            j += 1
            fname = os.path.join(dest_path, new_name)
            if edges:
                new_im = cv2.Canny(new_im, 100, 200)
            if transform:
                # 2x mirror
                img_flip_vert = np.flip(new_im, axis=0)
                filename = os.path.join(fname + '_vert.jpg')
                cv2.imwrite(filename, img_flip_vert)

                img_flip_horiz = np.flip(new_im, axis=1)
                filename = os.path.join(fname + '_horiz.jpg')
                cv2.imwrite(filename, img_flip_horiz)

                # rotate 3 times and save
                for i in range(1, 4):
                    img_flip_vert = cv2.rotate(img_flip_vert, cv2.ROTATE_90_CLOCKWISE)
                    filename = os.path.join(fname + '_' + str(i*90) + '_vert.jpg')
                    cv2.imwrite(filename, img_flip_vert)

                    img_flip_horiz = cv2.rotate(img_flip_horiz, cv2.ROTATE_90_CLOCKWISE)
                    filename = os.path.join(fname + '_' + str(i*90) + '_horiz.jpg')
                    cv2.imwrite(filename, img_flip_horiz)
            cv2.imwrite(os.path.join(fname+'.jpg'), new_im)
    csv_name = 'label_legend.csv'
    with open(csv_name, 'w') as csvfile:
        for key in legend.keys():
            csvfile.write("%d,%s\n"%(int(legend[key]), key))
            
def blackandwhite(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    (thresh, img) = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    return img / 255.0
            

In [ ]:
#copy dataset into dest_name folder and create a 'label legend'

dest_name = 'ninety-six'
desired_size = 96

#root_path = os.path.join(os.getcwd(), 'drive', 'My Drive', 'datasets')
root_path = os.path.join(os.getcwd(), 'datasets')
dest_path = os.path.join(root_path, dest_name)
process_data(os.path.join(root_path, '100 leaves plant species', 'data'), dest_path, desired_size, transform=False, edges=False)

In [ ]:
#Split processed dataset into training and validation

test_perc = .25
random_seed = 101
data_path = dest_path
image_files = os.listdir(dest_path)
data = [load_image(os.path.join(data_path, file)) for file in image_files]
data = [blackandwhite(img) for img in data]
labels = [int(file[:2]) for file in image_files]
train_images, test_images, train_labels, test_labels = train_test_split(data, labels, test_size=test_perc, random_state=random_seed)
test_label_name = extract_names(test_labels)
print("training labels: ", train_labels[:10])
print("testing labels: ", test_labels[:10])
print("testing names:", test_label_name[:10])

In [ ]:
preview_index = random.randint(0, len(train_images)-1)
plt.subplot(1,2,1)
plt.imshow(train_images[preview_index])

In [ ]:
train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)
print(type(train_images[0]), type(train_labels[0]))
print(train_labels[:15])
print(train_images.shape, train_labels.shape)
print(train_images[0].shape)

## CONVOLUTIONAL NET

In [ ]:
layers = [
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), padding="same", activation=tf.nn.relu, input_shape=train_images.shape[1:]),
    tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding="same", activation=tf.nn.relu),
    tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    #tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding="same", activation=tf.nn.relu),
    #tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    #tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation=tf.nn.relu),
    #tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    #tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation=tf.nn.relu),
    tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    tf.keras.layers.Flatten(),
    #tf.keras.layers.Dense(units=512, activation=tf.nn.relu),
    tf.keras.layers.Dense(units=256, activation=tf.nn.relu),
    tf.keras.layers.Dense(units=100, activation=tf.nn.softmax)
]

model = tf.keras.Sequential(layers)
model.compile(optimizer=tf.optimizers.Adam(),
              loss=tf.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.metrics.SparseCategoricalAccuracy()])
model.fit(train_images, train_labels, epochs=15, batch_size=50)
#loss, accuracy = model.Evaluate()
model.save_weights("model.tf")

In [ ]:
eval_model = tf.keras.Sequential(layers)
eval_model.load_weights("model.tf")
eval_predictions = eval_model.predict(np.expand_dims(test_images, axis=-1))
cols = 4
rows = np.ceil(len(test_images)/cols)
fig = plt.gcf()
fig.set_size_inches(cols * 4, rows * 4)
predicted_labels = np.argmax(eval_predictions, axis=1)
predicted_names = extract_names(predicted_labels)
predicted_correct = [1 if predicted_labels[i] ==  test_labels[i] else 0 for i in range(len(test_labels))  ]
accuracy = np.sum(predicted_correct) / len(test_labels)
print("Test Accuracy = ", accuracy)
for i in range(15):
    plt.subplot(rows, cols, i+1)
    plt.imshow(test_images[i], cmap="gray")
    title = predicted_names[i][10:] + " / " + test_label_name[i][10:]
    plt.title(title)
    plt.axis('off')

## KNN

In [ ]:
# Distance between instances x1 and x2
def dist(x1, x2):
    distance = x1 - x2
    return np.linalg.norm(distance)

# Predict label for instance x, using k nearest neighbors in training data
def classify_knn(train_x, train_y, k, x):
    # keep a list of all the distances with the values
    distances = []
    for i, data in enumerate(train_x):
        # caluclate distance between x and data
        distance = dist(x, data)
        # add distance and label to list
        distances.append((distance, train_y[i]))

    # sort list of distances
    distances.sort()

    # tally up the results of the first k items in the sorted list
    results = {}
    best = (None, 0)
    for i in range(k):
        distance, label = distances[i]
        # increment label by 1 if the label exists, or set to 1 if it doesn't
        results[label] = results.get(label, 0) + 1
        # keep track of the label with the most "votes"
        if (results[label] > best[1]):
            best = (label, results[label])

    return best[0]

# Run classifier and compute accuracy
def runTest(test_x, test_y, train_x, train_y, k):
    num_iters = test_x.shape[0]
    correct = 0
    i = 1
    for (x,y) in zip(test_x, test_y):
        clear_output(wait=True)
        print('progress: {:.2f}% complete'.format(100* i/num_iters))
        i += 1
        if classify_knn(train_x, train_y, k, x) == y:
            correct += 1
    acc = float(correct)/len(test_x)
    return acc

In [ ]:
runTest(test_images, test_labels, train_images, train_labels, 10)